In [ ]:
!pip install pynvml GPUtil

In [ ]:
import cv2
import numpy as np
import time
import torch
import os
from skimage.metrics import peak_signal_noise_ratio as psnr

base_path = "/content/drive/My Drive/dataset"

def median_filter_gpu(image_tensor, ksize=5):
    if ksize % 2 == 0:
        ksize += 1

    image_np = image_tensor.permute(1, 2, 0).cpu().numpy()
    image_np = (image_np * 255).astype(np.uint8)
    filtered_image_np = cv2.medianBlur(image_np, ksize)
    filtered_image_tensor = torch.tensor(filtered_image_np, dtype=torch.float32).permute(2, 0, 1).cuda()
    filtered_image_tensor = filtered_image_tensor / 255.0
    return filtered_image_tensor

def measure_metrics(image, filtered_image, inference_time):
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory_allocated = torch.cuda.memory_allocated() / (1024 * 1024)
    gpu_memory_reserved = torch.cuda.memory_reserved() / (1024 * 1024)
    gpu_load = torch.cuda.utilization(0)
    gpu_temperature = torch.cuda.temperature(0)
    psnr_value = psnr(image, filtered_image)

    metrics = {
        "GPU": gpu_name,
        "GPU Memory Usage (MB)": f"{gpu_memory_allocated:.2f} MB / {gpu_memory_reserved:.2f} MB",
        "GPU Load (%)": gpu_load,
        "Temperature (°C)": gpu_temperature,
        "Inference Time (ms)": inference_time,
        "PSNR (dB)": psnr_value,
    }

    return metrics

def process_image(image_path, output_path, ksize=5):
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)

    if image.shape[0] > 720 or image.shape[1] > 1280:
        raise ValueError("Image size exceeds the allowed dimensions of 1280x720p")

    image_tensor = torch.tensor(image, dtype=torch.float32).cuda()
    if len(image_tensor.shape) == 2:
        image_tensor = image_tensor.unsqueeze(0)
    else:
        image_tensor = image_tensor.permute(2, 0, 1)

    image_tensor = image_tensor / 255.0

    start_time = time.time()
    filtered_image_tensor = median_filter_gpu(image_tensor, ksize=ksize)
    inference_time = (time.time() - start_time) * 1000

    filtered_image = filtered_image_tensor.cpu().numpy()
    if len(filtered_image.shape) == 3:
        filtered_image = np.moveaxis(filtered_image, 0, -1)
    else:
        filtered_image = filtered_image.squeeze()
    filtered_image = (filtered_image * 255).astype(np.uint8)

    metrics = measure_metrics(image, filtered_image, inference_time)

    return metrics

def process_all_images_in_all_folders(folders, ksize=5):
    total_metrics = {
        "GPU": None,
        "GPU Memory Usage (MB)": "0.00 MB / 0.00 MB",
        "GPU Load (%)": 0,
        "Temperature (°C)": 0,
        "Inference Time (ms)": 0,
        "PSNR (dB)": 0,
    }

    total_images = 0
    total_time_start = time.time()

    for noisy_folder, denoised_folder in folders.items():
        input_folder = os.path.join(base_path, noisy_folder)
        output_folder = os.path.join(base_path, denoised_folder)

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        image_files = [f for f in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, f))]

        total_images += len(image_files)

        for image_file in image_files:
            image_path = os.path.join(input_folder, image_file)
            output_path = os.path.join(output_folder, image_file)

            metrics = process_image(image_path, output_path, ksize)

            total_metrics["Inference Time (ms)"] += metrics["Inference Time (ms)"]
            total_metrics["PSNR (dB)"] += metrics["PSNR (dB)"]
            total_metrics["GPU"] = metrics["GPU"]
            total_metrics["GPU Memory Usage (MB)"] = metrics["GPU Memory Usage (MB)"]
            total_metrics["GPU Load (%)"] += metrics["GPU Load (%)"]
            total_metrics["Temperature (°C)"] += metrics["Temperature (°C)"]

    total_time_end = time.time()
    total_time_taken = (total_time_end - total_time_start) * 1000

    if total_images > 0:
        avg_inference_time = total_metrics["Inference Time (ms)"] / total_images
        avg_gpu_load = total_metrics["GPU Load (%)"] / total_images
        avg_temperature = total_metrics["Temperature (°C)"] / total_images
        avg_psnr = total_metrics["PSNR (dB)"] / total_images

        total_metrics["Inference Time (ms)"] = avg_inference_time
        total_metrics["GPU Load (%)"] = avg_gpu_load
        total_metrics["Temperature (°C)"] = avg_temperature
        total_metrics["PSNR (dB)"] = avg_psnr

    print("Final Metrics Summary:")
    for metric, value in total_metrics.items():
        print(f"{metric}: {value}")
    print(f"Total Time Taken (ms): {total_time_taken:.2f}")
    print(f"Average Inference Time per Image (ms): {avg_inference_time:.2f}")

if __name__ == "__main__":
    folders = {
        "noisy5": "denoised5",
        "noisy15": "denoised15",
        "noisy35": "denoised35"
    }

    process_all_images_in_all_folders(folders, ksize=5)
